In [1]:
script_depth = '../../'

import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import os
import sys
sys.path.append(script_depth)
from user_packages import hashing


In [4]:
# set variables
target_model_name = 'invite.please.over'
source_file_name = 'mysql_tables.csv'
source_file_path = script_depth + 'imports/physical/mysql/'

record_source = 'MySQLScrape'
SystemKeyPhrase = 'DRS' # this should come from the System Hub


In [3]:
# db connect
conn = sqlite3.connect(script_depth + 'full_metadata.db')

In [14]:
# read table to df
df = pd.read_csv(os.path.join(source_file_path, source_file_name))
df = df.replace({np.nan: None})

#df

In [15]:
# add extra columns

# columns with variable data
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()
df['ModelKeyPhrase'] = target_model_name
df['SystemKeyPhrase'] = SystemKeyPhrase


# derive keyphrase columns
df['PhysicalStructureKeyPhrase'] = df.apply(
  lambda row: 
    row['SERVER_NAME']
    + '.' + row['DATABASE_NAME']
    + '.' + row['SCHEMA_NAME']
    + '.' + row['TABLE_NAME']
  , axis=1
)
df['ModelPhysicalStructureKeyPhrase'] = df.apply(  lambda row: row['ModelKeyPhrase'] + ':' + row['PhysicalStructureKeyPhrase'], axis=1)
df['SystemPhysicalStructureKeyPhrase'] = df.apply(  lambda row: row['SystemKeyPhrase'] + ':' + row['PhysicalStructureKeyPhrase'], axis=1)

# hash the keyphrases
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['ModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'SystemHashKey'
  , columns = ['SystemKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalStructureHashKey'
  , columns = ['PhysicalStructureKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelPhysicalStructureHashKey'
  , columns = ['ModelPhysicalStructureKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'SystemPhysicalStructureHashKey'
  , columns = ['SystemPhysicalStructureKeyPhrase']
)


# hash the payload
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = [
      'SERVER_NAME'
    , 'DATABASE_NAME'
    , 'SCHEMA_NAME'
    , 'TABLE_NAME'

    , 'TABLE_TYPE'
    , 'ENGINE'
    , 'ROW_COUNT'
    , 'AVG_ROW_LENGTH'
    , 'DATA_LENGTH'
    , 'INDEX_LENGTH'
    , 'AUTO_INCREMENT'
    , 'UPDATE_TIME'
    , 'COLUMN_COUNT'
    , 'TABLE_COLLATION'
    , 'TABLE_COMMENT'
  ]
)

#df

In [9]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_MysqlCsvToPhysicalStructure")
conn.commit()
df.to_sql('stg_Py_MysqlCsvToPhysicalStructure', conn, if_exists='append', index=False)

825

In [10]:
# Write to the Hub (PhysicalStructure)
sql_query = """
INSERT INTO rv_h_PhysicalStructure
(
    PhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , PhysicalStructureKeyPhrase
)
SELECT DISTINCT
    PhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , PhysicalStructureKeyPhrase
FROM
  stg_Py_MysqlCsvToPhysicalStructure
WHERE
  PhysicalStructureHashKey NOT IN (SELECT PhysicalStructureHashKey FROM rv_h_PhysicalStructure)
""";
conn.execute(sql_query)
conn.commit()

In [11]:
# Write to the Satellite (PhysicalStructure_Mysql)
sql_query = """
INSERT INTO rv_s_PhysicalStructure_Mysql
(
    PhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , HashDiff

  , SERVER_NAME
  , "DATABASE_NAME"
  , SCHEMA_NAME
  , TABLE_NAME

  , TABLE_TYPE
  , ENGINE
  , ROW_COUNT
  , AVG_ROW_LENGTH
  , DATA_LENGTH
  , INDEX_LENGTH
  , AUTO_INCREMENT
  , UPDATE_TIME
  , COLUMN_COUNT
  , TABLE_COLLATION
  , TABLE_COMMENT

)
SELECT DISTINCT
    stg.PhysicalStructureHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  
  , stg.SERVER_NAME
  , stg."DATABASE_NAME"
  , stg.SCHEMA_NAME
  , stg.TABLE_NAME

  , stg.TABLE_TYPE
  , stg.ENGINE
  , stg.ROW_COUNT
  , stg.AVG_ROW_LENGTH
  , stg.DATA_LENGTH
  , stg.INDEX_LENGTH
  , stg.AUTO_INCREMENT
  , stg.UPDATE_TIME
  , stg.COLUMN_COUNT
  , stg.TABLE_COLLATION
  , stg.TABLE_COMMENT

FROM
  stg_Py_MysqlCsvToPhysicalStructure AS stg
  LEFT OUTER JOIN rv_s_PhysicalStructure_Mysql AS sat ON (
    stg.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_PhysicalStructure_Mysql AS z
      WHERE z.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [12]:
# Write to the Link (Model - PhysicalStructure)
sql_query = """
INSERT INTO rv_l_ModelPhysicalStructure
(
  ModelPhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , ModelHashKey
  , PhysicalStructureHashKey
)
SELECT
  stg.ModelPhysicalStructureHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ModelHashKey
  , stg.PhysicalStructureHashKey
FROM
  stg_Py_MysqlCsvToPhysicalStructure stg
WHERE
  stg.ModelPhysicalStructureHashKey IS NOT NULL
  AND ModelPhysicalStructureHashKey NOT IN (
    SELECT ModelPhysicalStructureHashKey
    FROM rv_l_ModelPhysicalStructure
  )
""";
conn.execute(sql_query)
conn.commit()

In [13]:
# Write to the Link (System - PhysicalStructure)
sql_query = """
INSERT INTO rv_l_SystemPhysicalStructure
(
  SystemPhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , SystemHashKey
  , PhysicalStructureHashKey
)
SELECT
  stg.SystemPhysicalStructureHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.SystemHashKey
  , stg.PhysicalStructureHashKey
FROM
  stg_Py_MysqlCsvToPhysicalStructure stg
WHERE
  stg.SystemPhysicalStructureHashKey IS NOT NULL
  AND SystemPhysicalStructureHashKey NOT IN (
    SELECT SystemPhysicalStructureHashKey
    FROM rv_l_SystemPhysicalStructure
  )
""";
conn.execute(sql_query)
conn.commit()